In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix


In [4]:
# Đọc dữ liệu đơn hàng
orders = pd.read_csv('orders.xls')

# Đọc chi tiết sản phẩm đã mua trong đơn hàng
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')

# Đọc sản phẩm
products = pd.read_csv('products.csv')


In [5]:
# Chỉ lấy đơn hàng "prior"
orders_prior = orders[orders['eval_set'] == 'prior']

# Join đơn hàng với sản phẩm đã mua
prior_merged = pd.merge(order_products_prior, orders_prior, on='order_id')

# Chỉ lấy các cột cần thiết
user_product = prior_merged[['user_id', 'product_id']]

user_product.head()


,user_id,product_id
0,202279,33120
1,202279,28985
2,202279,9327
3,202279,45918
4,202279,30035


In [6]:
# Đếm số lần user mua sản phẩm
user_product_counts = user_product.groupby(['user_id', 'product_id']).size().reset_index(name='times_purchased')

user_product_counts.head()


,user_id,product_id,times_purchased
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [7]:
# Encode user_id và product_id thành chỉ số (index bắt đầu từ 0)
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

user_product_counts['user_idx'] = user_encoder.fit_transform(user_product_counts['user_id'])
user_product_counts['product_idx'] = product_encoder.fit_transform(user_product_counts['product_id'])

# Tạo sparse matrix
interaction_matrix = csr_matrix((
    user_product_counts['times_purchased'],
    (user_product_counts['user_idx'], user_product_counts['product_idx'])
))

interaction_matrix


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 13307953 stored elements and shape (206209, 49677)>

In [8]:
print('Number of users:', interaction_matrix.shape[0])
print('Number of products:', interaction_matrix.shape[1])


Number of users: 206209
Number of products: 49677


In [9]:
import implicit

# Dùng GPU nếu có (cuda), còn không thì dùng CPU
model = implicit.als.AlternatingLeastSquares(
    factors=50,          # số lượng latent factors (embedding size)
    regularization=0.01, # hệ số regularization để tránh overfitting
    iterations=20,       # số vòng lặp
    random_state=42
)

# implicit yêu cầu chuyển data thành item-user matrix (transpose)
item_user_matrix = interaction_matrix.T

# Fit model
model.fit(item_user_matrix)


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 20 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.19024324417114258 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
target_user_id = 1
user_idx = user_encoder.transform([target_user_id])[0]

# Lấy sparse vector user
user_items_single = item_user_matrix.T[user_idx]

# Recommend
recommended = model.recommend(
    userid=0,
    user_items=user_items_single,
    N=10,
    filter_already_liked_items=True
)




In [11]:
# Lấy item_idx và ép kiểu int
recommended_product_idxs = [int(item[0]) for item in recommended]

# Decode product_idx → product_id
recommended_product_ids = product_encoder.inverse_transform(recommended_product_idxs)

# Lấy tên sản phẩm
recommended_products = products[products['product_id'].isin(recommended_product_ids)]

print(recommended_products[['product_id', 'product_name']])

      product_id                product_name
0              1  Chocolate Sandwich Cookies
5589        5590              Oatmeal Cereal


In [12]:
import pickle

# Lưu model
with open('als_model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [13]:
# Tải lại model
with open('als_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Kiểm tra model đã tải lại có thể dự đoán
recommended = loaded_model.recommend(
    userid=0,
    user_items=user_items_single,
    N=10,
    filter_already_liked_items=True
)

# In ra các sản phẩm được đề xuất
recommended_product_idxs = [int(item[0]) for item in recommended]
recommended_product_ids = product_encoder.inverse_transform(recommended_product_idxs)
recommended_products = products[products['product_id'].isin(recommended_product_ids)]

print(recommended_products[['product_id', 'product_name']])


      product_id                product_name
0              1  Chocolate Sandwich Cookies
5589        5590              Oatmeal Cereal


In [14]:
import pickle
from scipy.sparse import save_npz

# --- Thêm đoạn code này ---
# Lưu user encoder
with open('user_encoder.pkl', 'wb') as f:
    pickle.dump(user_encoder, f)

# Lưu product encoder
with open('product_encoder.pkl', 'wb') as f:
    pickle.dump(product_encoder, f)

# Lưu user-item interaction matrix (cần cho model.recommend)
# Lưu ý: Đây là ma trận user-item gốc, không phải ma trận đã transpose (item-user) dùng để fit
save_npz('interaction_matrix.npz', interaction_matrix)
# --- Kết thúc đoạn code thêm ---

